In [34]:
import time
import re
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
# driver.implicitly_wait(10)

In [4]:
driver.get('https://hsreplay.net/meta/#tab=archetypes')
time.sleep(10)

# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "myDynamicElement"))
#     )
# finally:
#     driver.quit()

In [5]:
driver.execute_script('return document.readyState;')

'complete'

In [8]:
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML, 'html.parser')

In [ ]:
print(soup.prettify())

In [51]:
all_decks = soup.find_all("a",class_="table-cell", attrs={"aria-describedby": re.compile("column0")})

In [11]:
all_decks[0]

<a aria-describedby="table1-row0 table1-column0" class="table-cell" href="/archetypes/267/gonk-druid" role="gridcell" style="height: 36px; left: 0px; position: absolute; top: 0px; width: 174px; color: rgb(255, 80, 80); line-height: 36px; background: white;">37.7%</a>

In [12]:
all_decks[0].get('href')

'/archetypes/267/gonk-druid'

In [52]:
archetypes = list(map(lambda tag: (tag.get('href'), tag.contents[0].rstrip('%')), all_decks))

In [53]:
archetypes_f = list(filter(lambda archetype: archetype[0] is not None, archetypes))

In [54]:
links = [x[0] for x in archetypes_f]

In [55]:
links

['/archetypes/267/gonk-druid',
 '/archetypes/6/malygos-druid',
 '/archetypes/247/mechathun-druid',
 '/archetypes/226/mill-druid',
 '/archetypes/205/spiteful-druid',
 '/archetypes/149/taunt-druid',
 '/archetypes/7/token-druid',
 '/archetypes/270/undatakah-druid',
 '/archetypes/139/deathrattle-hunter',
 '/archetypes/137/midrange-hunter',
 '/archetypes/212/odd-hunter',
 '/archetypes/180/recruit-hunter',
 '/archetypes/138/secret-hunter',
 '/archetypes/179/spell-hunter',
 '/archetypes/182/big-spell-mage',
 '/archetypes/145/elemental-mage',
 '/archetypes/1/freeze-mage',
 '/archetypes/214/odd-mage',
 '/archetypes/142/quest-mage',
 '/archetypes/273/secret-odd-mage',
 '/archetypes/11/tempo-mage',
 '/archetypes/261/deathrattle-paladin',
 '/archetypes/215/even-paladin',
 '/archetypes/268/holy-wrath-paladin',
 '/archetypes/249/mech-paladin',
 '/archetypes/168/otk-dk-paladin',
 '/archetypes/216/odd-paladin',
 '/archetypes/15/secret-paladin',
 '/archetypes/228/clone-priest',
 '/archetypes/206/combo-

In [17]:
all_decks[0].contents[0].rstrip('%')

'37.7'

In [18]:
deck_name_extract = re.compile('[^\/]+$')

In [24]:
deck = deck_name_extract.search(links[2]).group()

In [25]:
deck.replace('-', ' ').title()

'Mechathun Druid'

In [56]:
base_url = 'https://hsreplay.net'
winrates = {}
df_wr = pd.DataFrame()

for link in links[:3]:
    deck = deck_name_extract.search(link).group()
    url = base_url + link + '#tab=matchups'
    print(f'{deck}:{url}')
    driver.get(url)
    time.sleep(5)
    innerHTML = driver.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML, 'html.parser')
    matchups = soup.find_all("a",class_="table-cell", attrs={"aria-describedby": re.compile("column0")})
    winrates[deck] = list(map(lambda matchup: (float(matchup.contents[0].rstrip('%')), deck_name_extract.search(matchup.get('href')).group()), matchups))
    
    zlst = list(zip(*winrates[deck]))
    s_wr = pd.Series(zlst[0], index=zlst[1])
    
    df_wr[deck] = s_wr

gonk-druid:https://hsreplay.net/archetypes/267/gonk-druid#tab=matchups
malygos-druid:https://hsreplay.net/archetypes/6/malygos-druid#tab=matchups
mechathun-druid:https://hsreplay.net/archetypes/247/mechathun-druid#tab=matchups


In [57]:
def clean_names(names):
    return list(map(lambda deck:deck.replace('-', ' ').title(), list(names)))

In [58]:
clean_names(df_wr.columns.values)

['Gonk Druid', 'Malygos Druid', 'Mechathun Druid']

In [59]:
clean_names(df_wr.index.values)

['Mechathun Druid',
 'Deathrattle Hunter',
 'Midrange Hunter',
 'Secret Hunter',
 'Spell Hunter',
 'Big Spell Mage',
 'Odd Mage',
 'Tempo Mage',
 'Even Paladin',
 'Otk Dk Paladin',
 'Odd Paladin',
 'Clone Priest',
 'Combo Priest',
 'Control Priest',
 'Odd Rogue',
 'Tempo Rogue',
 'Thief Rogue',
 'Even Shaman',
 'Control Warlock',
 'Cube Warlock',
 'Even Warlock',
 'Zoo Warlock',
 'Odd Quest Warrior',
 'Odd Warrior']

In [38]:
list(df_wr.index.values)

['mechathun-druid',
 'deathrattle-hunter',
 'midrange-hunter',
 'secret-hunter',
 'spell-hunter',
 'big-spell-mage',
 'odd-mage',
 'tempo-mage',
 'even-paladin',
 'otk-dk-paladin',
 'odd-paladin',
 'clone-priest',
 'combo-priest',
 'control-priest',
 'odd-rogue',
 'tempo-rogue',
 'thief-rogue',
 'even-shaman',
 'control-warlock',
 'cube-warlock',
 'even-warlock',
 'zoo-warlock',
 'odd-quest-warrior',
 'odd-warrior']

In [41]:
df_wr.index = list(map(lambda deck:deck.replace('-', ' ').title(), list(df_wr.index.values)))

In [47]:
clean_names(df_wr.columns.values)

['Gonk Druid', 'Malygos Druid', 'Mechathun Druid']

In [42]:
df_wr

,Gonk Druid,Malygos Druid,Mechathun Druid
Mechathun Druid,38.1,40.4,50.0
Deathrattle Hunter,30.1,30.4,28.4
Midrange Hunter,36.9,37.1,41.9
Secret Hunter,25.4,37.7,35.7
Spell Hunter,22.8,39.3,48.9
Big Spell Mage,39.5,50.0,45.3
Odd Mage,29.3,51.1,61.3
Tempo Mage,47.4,NaN,35.1
Even Paladin,28.2,30.6,47.2
Otk Dk Paladin,34.0,48.1,73.3


In [36]:
vfunc = np.vectorize(clean_name)

In [37]:
clean_name(df_wr.index.values)

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [60]:
driver.quit()

In [16]:
driver.quit()

df_wr = df_wr.T
df_wr = df_wr[list(df_wr.index.values)]
df_wr.to_csv(f"hsreplay_winrates_{datetime.today().strftime('%Y%m%d')}.csv")

In [ ]:
# File approach
# for i, line in enumerate(lines):
#     if line.startswith('Versus'):
#         matchups.append((' '.join(lines[i-1].split()),' '.join(lines[i].split()[1:]), float(' '.join(lines[i+1].split()[1:]).rstrip('%'))))